In [1]:
import pandas as pd
import json

In [2]:
FILE = "../data/AMAZON_FASHION_5.json"

In [3]:
df = pd.DataFrame()

In [62]:
with open(FILE, 'r') as f:
    for json_line in f.readlines():
        line = json.loads(json_line)
        if line.get("style"):
            for key, value in line.pop("style").items():
                line[f"style.{key}"] = value
        df = df.append(line, ignore_index=True)

In [63]:
reviews_by_user = df.groupby(by="reviewerID").count()

In [64]:
reviews_by_user.max()

overall             16
verified            16
reviewTime          16
asin                16
reviewerName        16
reviewText          16
summary             16
unixReviewTime      16
style.Size:         16
style.Color:        16
style.Size Name:     1
style.Style:         1
vote                16
image               16
dtype: int64

In [65]:
reviews_by_user.min()

overall             4
verified            4
reviewTime          4
asin                4
reviewerName        4
reviewText          0
summary             4
unixReviewTime      4
style.Size:         0
style.Color:        0
style.Size Name:    0
style.Style:        0
vote                0
image               0
dtype: int64

In [66]:
df.to_json("../data/AMAZON_FASION_5_formatted.json")

In [32]:
df = pd.read_json("../data/AMAZON_FASION_5_formatted.json")

In [33]:
df

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style.Size:,style.Color:,style.Size Name:,style.Style:,vote,image
0,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue/Orange,None,None,NaN,None
1,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Black (37467610) / Red/White,None,None,NaN,None
2,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue/Gray Logo,None,None,NaN,None
3,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue (37867638-99) / Yellow,None,None,NaN,None
4,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue/Pink,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3171,5,1,"07 2, 2018",A2077NII5H62R2,B005AGO4LU,Amazon Customer,Perfect fit!,Five Stars,1530489600,8.5 B(M) US,Green Glow/Seaweed - Hasta - White,None,None,NaN,None
3172,5,1,"06 28, 2018",A2IBS6PIPAGAB5,B005AGO4LU,J. Avila,My favorite cross trainers!,Comfortable,1530144000,5 B(M) US,Wolf Grey/Black-pink Blast/White,None,None,NaN,None
3173,5,1,"06 25, 2018",A1GTC5EVSJNCQ8,B005AGO4LU,Amazon Customer,Love them fit perfect,Five Stars,1529884800,8 B(M) US,Blue Tint/Green Glow/Hasta/White,None,None,NaN,None
3174,5,1,"06 20, 2018",A311XHHLM12MUT,B005AGO4LU,Peter,Favorite Nike shoe ever! The flex sole is exce...,Love them!,1529452800,9 B(M) US,Blue Tint/Green Glow/Hasta/White,None,None,NaN,None


In [25]:
df = df[["reviewerID", "overall", "asin"]]

In [26]:
df.drop_duplicates(inplace=True)

In [27]:
df.reset_index(inplace=True)

In [28]:
df = df.pivot_table(index='reviewerID', columns='asin', values='overall')

# Changing the sparsity of the matrix

This could be done either by looping of each row until the desired amount of dropped cells is reached (implemented) or we could choose rows at random and drop cells untill the desired amount is reached. This way not only the first X rows will drop more than the others, but it would be divided

In [29]:
import numpy as np

In [30]:
sparsity = lambda dataf: dataf.isna().sum().sum() / (dataf.shape[0] * dataf.shape[1])

In [31]:
total = df.shape[0] * df.shape[1]

In [16]:
current_nan = df.isnull().sum().sum()

In [18]:
sparsity(df)

0.7583823295725409

In [19]:
desired_percentage = .8

In [20]:
cells_to_drop = (desired_percentage * total) - current_nan

In [21]:
cells_dropped = 0
while True:
    for index, row in df.iterrows():
        nans = np.where(row.notnull())[0]
        if not len(nans):
            continue
        drop = np.random.choice(nans)
        row[drop] = np.nan
        cells_dropped += 1
        if cells_dropped >= cells_to_drop:
            break    
    if cells_dropped >= cells_to_drop:
        break        


In [22]:
cells_dropped

524

In [23]:
sparsity(df)

0.8000158906721754